# Análisis Exploratorio

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
# base de datos de vuelos preprocesada
df = pd.read_parquet("../Preprocessing/flightsFilteredCleaned.parquet")

# Añadimos el FlightNum como variable categórica
df['FLIGHT_NUMBER']=df['FLIGHT_NUMBER'].astype(object) 

pd.set_option('display.max_columns', None)
df.head()

Primero hacemos un análisis de los retrasos en EEUU a lo largo del año

In [ ]:
data = pd.DataFrame()
data["Total"] = df.groupby('DATE')["FLIGHT_NUMBER"].count()
data["Delayed"] = df[df["ARRIVAL_DELAY"]>0].groupby('DATE')["FLIGHT_NUMBER"].count()
data.head()

In [ ]:
fig = px.line(data, x=data.index, y=["Total", "Delayed"], color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_layout(
    title="Trend in total and delayed flights throughout the year",
    xaxis_title="Date",
    yaxis_title="Flights",
    legend_title="Number of flights",
    template="plotly_dark",
    hovermode="x unified"
)

fig.add_hline(y=np.mean(data["Total"]), line_dash="dash", line_color="white", annotation_text="mean", annotation_position="top right")
fig.add_hline(y=np.mean(data["Delayed"]), line_dash="dash", line_color="white", annotation_text="mean", annotation_position="bottom right")

fig.show()

En los meses de verano los vuelos realizados son superiores a la media, como cabría esperar. Pero lo interesante aparece al observar los retrasos, no solo repuntan en esas fechas sino también a final de año. Podríamos plantear la hipótesis inicial de que en fechas de mayores vuelos hay más retrasos debido a la falta de capacidad en ciertos aeropuertos.
<br><br>Además, parece haber una tendencia semanal, vamos a confirmarlo:

In [ ]:
## Definimos la paleta usada con plotly express, y la utilizaremos con plotly go
color_palette = px.colors.qualitative.Vivid

In [ ]:
data = pd.DataFrame()
data["Total"] = df.groupby(df['DATE'].dt.day_name())["FLIGHT_NUMBER"].count()
data["Delayed"] = df[df["ARRIVAL_DELAY"]>0].groupby(df['DATE'].dt.day_name())["FLIGHT_NUMBER"].count()
data = data.reindex(index = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
data.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=data.index, y=data["Total"], name="Total", marker_color = color_palette[0]))
fig.add_trace(go.Scatter(x=data.index, y=data["Delayed"], name="Delayed", line_color=color_palette[1]))
fig.update_layout(
    title="Total and delayed flights by airline",
    xaxis_title="Weekday",
    yaxis_title="Flights",
    legend_title="Number of Flights",
    template="plotly_dark",
    hovermode="x unified"
)
fig.show()

Efectivamente, de media los sábados hay menos vuelos. Esto tiene sentido, ya que es raro volar en medio del fin de semana.

Lo siguiente será analizar cuánto se retrasan de media los vuelos y debido a qué causas, a lo largo del año

In [ ]:
data = df.groupby(df['DATE'].dt.month_name()).mean()
ORDERED_MONTHS = ["January", "February", "March", "April", "May", "June",
      "July", "August", "September", "October", "November", "December"]
data = data.reindex(index = ORDERED_MONTHS)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data["AIR_SYSTEM_DELAY"], stackgroup='one', name="Air system", line_color=color_palette[0]))
fig.add_trace(go.Scatter(x=data.index, y=data["SECURITY_DELAY"], stackgroup='one', name="Security", line_color=color_palette[1])) 
fig.add_trace(go.Scatter(x=data.index, y=data["AIRLINE_DELAY"], stackgroup='one', name="Airline", line_color=color_palette[2])) 
fig.add_trace(go.Scatter(x=data.index, y=data["LATE_AIRCRAFT_DELAY"], stackgroup='one', name="Late aircraft", line_color=color_palette[3])) 
fig.add_trace(go.Scatter(x=data.index, y=data["WEATHER_DELAY"], stackgroup='one', name="Weather", line_color=color_palette[4])) 
fig.add_trace(go.Scatter(x=data.index, y=data["OTHER_DELAY"], stackgroup='one', name="Other", line_color=color_palette[5])) 

fig.update_layout(
    title="Average delay and cause over the year",
    xaxis_title="Month",
    yaxis_title="Minutes",
    legend_title="Cause of delay",
    template="plotly_dark",
    hovermode="x unified"
)

fig.show()

Observamos que en general el retraso medio repunta en los meses de verano y de final e inicio de año, datos acordes al número de retrasos elevados que veíamos en esos meses. <br><br>
Además, la distribución de causas de los retrasos parece mantenerse relativamente estable a lo largo del año.

### ¿Correlación entre distancia y retraso?

In [ ]:
fig = px.scatter(df[df["ARRIVAL_DELAY"]>0], x="ARRIVAL_DELAY", y="DISTANCE", color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_layout(
    title="Tendencia vuelos totales y retrasados a lo largo del año",
    xaxis_title="Fecha",
    yaxis_title="Vuelos",
    legend_title="Leyenda",
    template="plotly_dark"
)
fig.show()

No hay correlación entre la distancia y el retraso de los vuelos. Esto lleva a pensar que la causa del retraso estará más relacionada con el despegue y aterrizaje del avión, que con un problema durante el trayecto.

## Análisis por Aeropuerto

Para hacer un análisis por aeropuertos, primero debemos agrupar los datos que nos interesan por aeropuerto

In [ ]:
# Agrupamos por aeropuertos
variables_to_group_by = ["ORIGIN_AIRPORT","ORIGIN_AIRPORT_NAME","ORIGIN_CITY","ORIGIN_STATE"]
airports = df.groupby(variables_to_group_by)\
             .agg({'ARRIVAL_DELAY':'mean','AIR_SYSTEM_DELAY':'mean',
                    'SECURITY_DELAY':'mean','AIRLINE_DELAY':'mean','LATE_AIRCRAFT_DELAY':'mean',
                    'WEATHER_DELAY':'mean','OTHER_DELAY':'mean', 'ORIGIN_LATITUDE':'mean',
                    'ORIGIN_LONGITUDE':'mean', 'FLIGHT_NUMBER':'count'})
airports = airports.rename(columns={"FLIGHT_NUMBER": "FLIGHTS"})
airports["DELAYED_FLIGHTS"] = df[df["ARRIVAL_DELAY"]>0].groupby(variables_to_group_by).size()
airports["DELAY_PERCENTAGE"] = airports["DELAYED_FLIGHTS"]/airports["FLIGHTS"]
airports = airports.sort_values("FLIGHTS",ascending=False).reset_index()
airports.head()

Será interesante ver cuantos vuelos salen de cada aeropuerto y cuantos de ellos se retrasan. Esto lo podemos analizar con las dos próximas visualizaciones.

In [ ]:
# Observamos el tamaño de los aeropuertos
fig = px.treemap(airports, path=["ORIGIN_AIRPORT"], values='FLIGHTS',
                  color='DELAY_PERCENTAGE', hover_data=['ORIGIN_CITY','ORIGIN_AIRPORT_NAME'],
                  color_continuous_scale='RdYlGn_r',
                  template="plotly_dark")
fig.update_layout(title="Departure Flights by Airport",
                  margin = dict(t=75, l=25, r=25, b=25))
fig.update_coloraxes(colorbar_tickformat = ',.2%', colorbar_title="Delayed Flights %")
fig.show()

Aquí podemos observar que entre los 16 aeropuertos con más vuelos nacionales, claramente los de Atlanta, Chicago y Dallas tienen un volumen de vuelos bastante superior al resto. <br><br>
Es curioso que en general los aeropuertos de menor tamaño tienen menos porcentaje de retrasos, a excepción de Atlanta, cuya cifra porcentual de retrasos es la más baja. Esta información es muy util, quizás en un futuro nos interese tomar este aeropuerto como referencia de buenas prácticas.

In [ ]:
fig = px.scatter_geo(airports, lat="ORIGIN_LATITUDE", lon = "ORIGIN_LONGITUDE",
                     size= "FLIGHTS", # size of markers
                     size_max= 30,
                     color= "ARRIVAL_DELAY", # which column to use to set the color of markers
                     scope="usa",
                     text = "ORIGIN_AIRPORT",
                     hover_data  = ['ORIGIN_CITY','ORIGIN_AIRPORT_NAME'],
                     color_continuous_scale='RdYlGn_r',
                     template="plotly_dark")
fig.update_traces(textposition="top center")
fig.update_layout(
    title="Origin airports with number of departing flights and percentage of delayed flights <br><br><sup>Size indicates the number of departing flights</sup>",
    legend_title="Causa del Retraso"
)
fig.update_coloraxes(colorbar_title="Average Delay")
fig.show()

Este mapa nos otorga información sobre la localización de estos dieciseis aeropuertos y su comparativa de retraso medio de vuelos de salida. Además de su tamaño, que ya conocíamos previamente. <br><br>
Es interesante observar como Atlanta tiene un retraso medio de los vuelos bastante bajo en comparación con el resto, por lo que parece ser sin duda una referencia para buenas prácticas en el sector.

### Estudiamos la disvisión de los retrasos en cada aeropuerto

In [ ]:
fig = make_subplots(rows=4, cols=4,
                    specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}],
                           [{"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}],
                           [{"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}],
                           [{"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}]],
                    horizontal_spacing = 0.03, vertical_spacing = 0.04)

delay_labels = ["AIR_SYSTEM_DELAY","SECURITY_DELAY","AIRLINE_DELAY","LATE_AIRCRAFT_DELAY","WEATHER_DELAY","OTHER_DELAY"]

for i in range(4):
    values1 = airports[delay_labels].iloc[i]
    fig.add_trace(go.Pie(labels=delay_labels, values=values1, direction ='clockwise', marker_colors=px.colors.qualitative.Vivid, hole=.33, 
                            title="%s:<br>%.3f s" % (airports["ORIGIN_AIRPORT"].iloc[i],np.sum(airports[delay_labels].iloc[i]))),
              row=1, col=i+1)
    values2 = airports[delay_labels].iloc[i+4]
    fig.add_trace(go.Pie(labels=delay_labels, values=values2, direction ='clockwise', marker_colors=px.colors.qualitative.Vivid, hole=.33, 
                            title="%s:<br>%.3f s" % (airports["ORIGIN_AIRPORT"].iloc[i+4],np.sum(airports[delay_labels].iloc[i+4]))),
              row=2, col=i+1)
    values3 = airports[delay_labels].iloc[i+8]
    fig.add_trace(go.Pie(labels=delay_labels, values=values3, direction ='clockwise', marker_colors=px.colors.qualitative.Vivid, hole=.33, 
                            title="%s:<br>%.3f s" % (airports["ORIGIN_AIRPORT"].iloc[i+8],np.sum(airports[delay_labels].iloc[i+8]))),
              row=3, col=i+1)
    values4 = airports[delay_labels].iloc[i+12]
    fig.add_trace(go.Pie(labels=delay_labels, values=values4, direction ='clockwise', marker_colors=px.colors.qualitative.Vivid, hole=.33, 
                            title="%s:<br>%.3f s" % (airports["ORIGIN_AIRPORT"].iloc[i+12],np.sum(airports[delay_labels].iloc[i+12]))),
              row=4, col=i+1)

fig.update_layout(title_text="Average Delay Distribution by Airport<br><sub>Values in seconds<sub>", 
                  legend_title="Delay Cause", template="plotly_dark", height=1200, width=1200,
                  legend=dict(orientation="h", y=-0.02, x =0.08))
fig.update_annotations(yshift=-125)

fig.show()

Vemos que aquí el retraso medio de cada aeropuerto es mayor que en el anterior mapa, esto es porque aquí tenemos en cuenta solo los vuelos retrasados, y por lo tanto la suma de los retrasos se divide entre menos vuelos.

La razón que más tiempo de retraso causa varía entre aeropuertos, pero está claro que el retraso de la aerolínea y los aviones tardíos son lo que más retraso causa en los vuelos. Eiste bastante homogeneidad en los resultados de los aeropuertos, por lo que podremos aplicar una estrategia similar a la hora de afrontar el plan de mejora.

### Por ultimo realizamos un estudio de los retrasos de los aeropuertos en función de la época del año

In [ ]:
fig = px.density_heatmap(df, y=df["DATE"].dt.month_name(), x=df["ORIGIN_AIRPORT"], z='ARRIVAL_DELAY', histfunc="avg",
                         color_continuous_scale='RdYlGn_r', text_auto=".2f")

fig.update_layout(
    title="Average Delay by Airport and Month",
    xaxis_title="Month",
    yaxis_title="Airport",
    template="plotly_dark",
    width = 1200, 
    height = 650
)

fig.layout.coloraxis.colorbar.title = 'Average Delay'

fig.show()

De aquí podemos sacar información mucho más personalizada para cada aeropuerto. Por ejemplo tanto ORD como BOS han de tener más cuidado con los retrasos en los dos primeros meses del año, mientras que aeropuertos como DFW o IAH empeoran su retraso medio al acercarse el verano.

# FIN